In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv('data/train.csv', encoding = 'Big5')
raw_data = raw_data.drop(['日期', '測站', '測項'], axis = 1)
# raw_data = raw_data.drop(['日期', '測站'], axis = 1)
raw_data = raw_data.replace('NR', -1)
raw_data = raw_data.astype(np.float)


raw_data

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,14.00,14.00,14.00,13.00,12.00,12.00,12.00,12.00,15.00,17.00,...,22.00,22.00,21.00,19.00,17.00,16.00,15.00,15.00,15.00,15.00
1,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,...,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80,1.80
2,0.51,0.41,0.39,0.37,0.35,0.30,0.37,0.47,0.78,0.74,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,0.20,0.15,0.13,0.12,0.11,0.06,0.10,0.13,0.26,0.23,...,0.10,0.13,0.14,0.23,0.18,0.12,0.10,0.09,0.10,0.08
4,0.90,0.60,0.50,1.70,1.80,1.50,1.90,2.20,6.60,7.90,...,2.50,2.20,2.50,2.30,2.10,1.90,1.50,1.60,1.80,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,1.80,1.80,1.80,1.80,1.80,1.70,1.70,1.80,1.80,1.80,...,1.80,1.80,2.00,2.10,2.00,1.90,1.90,1.90,2.00,2.00
4316,46.00,13.00,61.00,44.00,55.00,68.00,66.00,70.00,66.00,85.00,...,59.00,308.00,327.00,21.00,100.00,109.00,108.00,114.00,108.00,109.00
4317,36.00,55.00,72.00,327.00,74.00,52.00,59.00,83.00,106.00,105.00,...,18.00,311.00,52.00,54.00,121.00,97.00,107.00,118.00,100.00,105.00
4318,1.90,2.40,1.90,2.80,2.30,1.90,2.10,3.70,2.80,3.80,...,2.30,2.60,1.30,1.00,1.50,1.00,1.70,1.50,2.00,2.00


In [3]:
# raw_data.at[10, '1']
#raw_data.loc[10] 一行的数据 + label
ITEM_NUM    = 18 # CO CH4 ... WS_HR 一共 18 个测项
PM2_5_INDEX = 9 # pm2.5 的数据都在每一天数据的第 10 行
HOUR_NUM    = 9

ok,数据需要进行预处理。在本次模型中，我们就只取前九个时刻的 ```pm2.5``` 数值。


所以我只取了所有 ```pm2.5``` 的数据。


同时进行特征缩放，可以使用归一化.

In [4]:
all_pm2_5 = raw_data.loc[[x for x in range(raw_data.shape[0]) if x % ITEM_NUM == PM2_5_INDEX]]
all_pm2_5.reset_index(drop = True, inplace = True)
all_pm2_5 = (all_pm2_5 - all_pm2_5.mean()) / all_pm2_5.std() # 默认以 axis = 1 也就是一列方向求 mean/std
# print(all_pm2_5[:2])
# all_pm2_5[:2].mean().shape # 默认以 axis = 1 也就是一列方向求 mean
all_pm2_5

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.452531,1.327374,1.191365,1.191383,1.042314,0.853624,0.675466,0.275258,0.133550,0.713493,...,0.534408,1.056385,0.844019,1.188095,1.035038,1.020061,0.946991,0.407612,0.165562,-0.471103
1,0.122818,0.302828,0.802348,0.865200,0.410431,0.139792,-0.202396,-0.238557,-0.430003,0.190428,...,1.481168,0.940432,0.901598,0.969678,1.090511,0.362546,-0.459083,-0.222420,0.039219,0.382959
2,-0.009067,0.430896,0.607839,0.212834,-0.010824,-0.145741,-0.056086,-0.605567,-0.923112,-0.659552,...,0.311642,0.534594,0.383383,0.969678,0.757672,0.855682,-0.065382,0.350336,0.102390,1.105627
3,0.518473,0.558965,-0.235031,0.212834,0.410431,0.568091,0.748621,1.229484,2.176431,2.021154,...,1.982393,1.636153,1.650131,2.170970,2.865655,3.156983,3.702897,2.985016,4.019012,4.061995
4,4.013430,3.952774,3.784813,4.192267,4.131518,3.566185,3.967446,3.578349,3.444425,2.478835,...,2.093777,2.679735,1.707711,1.624928,1.478824,2.444675,2.578038,2.125881,2.945100,3.010842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,-0.932263,-0.593650,-0.040523,0.604254,1.182732,1.139157,0.602311,-0.018351,-0.077782,0.125045,...,-1.024960,-0.624941,0.037906,0.532845,0.702199,0.581718,0.665776,0.464887,0.291904,-0.076920
236,0.122818,0.494931,0.543003,0.930437,0.901895,0.353942,-0.202396,-0.238557,0.274439,0.255811,...,-0.802193,-1.204709,-1.344000,-0.886862,-0.851052,-1.116861,-1.358971,-0.967004,-0.971522,-0.996679
237,-1.261976,-1.041889,-1.013065,-0.896188,-0.923543,-0.930956,-0.348706,-0.238557,-0.218670,-0.724935,...,0.255950,0.244710,0.498542,0.205221,0.258413,0.472132,0.834505,0.751266,0.165562,0.185868
238,-0.338780,-0.145411,-0.299867,-0.178585,0.340222,0.211175,0.529155,0.128454,-0.007338,0.321194,...,-0.300967,-0.335057,-0.480309,-0.450029,-0.074427,-0.349761,-0.740298,-1.310658,-0.908351,-0.865285


现在数据预处理，构造一个 ```shape = (..., 9)``` 的 ```train set```。和对应行数的 ```y_train```。


```x_train``` 处理的顺序为 ```0 - 8, 1 - 9, 2 - 10 ...``` 以此类推

In [5]:
def train_data_treatment(raw_data):
    # 创建一个空的
    x_train = np.array([]).reshape(0, HOUR_NUM) 
    y_train = np.array([]).reshape(0, 1)
    for i in range(0, all_pm2_5.shape[1] - HOUR_NUM):
        nine_hours_pm2_5 = all_pm2_5[[ str(x) for x in range(i, i + HOUR_NUM) ]].values
        next_hour_pm2_5  = all_pm2_5[[ str(i + HOUR_NUM) ]].values
        x_train = np.append(x_train, nine_hours_pm2_5, axis = 0)
        y_train = np.append(y_train, next_hour_pm2_5, axis = 0)
    return x_train, np.squeeze(y_train)
    
x_train, y_train = train_data_treatment(all_pm2_5)
print(x_train)
print(y_train)

[[ 0.45253088  1.32737449  1.19136498 ...  0.67546569  0.27525762
   0.13355036]
 [ 0.12281803  0.30282815  0.80234785 ... -0.20239586 -0.2385566
  -0.43000281]
 [-0.0090671   0.43089644  0.60783928 ... -0.0560856  -0.60556676
  -0.92311183]
 ...
 [ 0.25594988  0.24471032  0.49854207 ...  0.8345052   0.75126567
   0.16556152]
 [-0.30096736 -0.33505747 -0.48030858 ... -0.74029821 -1.3106579
  -0.90835103]
 [-1.41480183 -1.14673238 -0.19241133 ...  0.21583243 -0.33697177
   0.16556152]]
[ 0.71349261  0.19042815 -0.65955159 ...  0.18586797 -0.86528519
  0.5800504 ]


接下来就是 train set 与 validation set 的分割

我们可以自己决定一个比例，我就定个 ```0.1``` 。(train : vali = 9 : 1) 

有效集的意义在于，检验模型在 train data 上的表现是否好的。

In [6]:
def split_train_valid(x_train, y_train, valid_percent = 0.1):
    sample_num  = x_train.shape[0]
    feature_num = x_train.shape[1]
    valid_num   = int(sample_num * 0.1) # 切片得整数
    order       = np.arange(sample_num)
    np.random.shuffle(order)
    x_valid_set, x_train_set = x_train[order][ : valid_num ], x_train[order][ valid_num : ]
    y_valid_set, y_train_set = y_train[order][ : valid_num ], y_train[order][ valid_num : ]

    return x_train_set, y_train_set, x_valid_set, y_valid_set
    
x_train_set, y_train_set, x_valid_set, y_valid_set = split_train_valid(x_train, y_train)
print(x_train_set)
print(y_train_set)

[[ 0.14759782 -0.36186984 -0.35989024 ...  0.1250451   0.03656978
   0.00512328]
 [ 0.41333071  0.27807103 -0.0810331  ...  0.20399451  0.45196038
   0.58895244]
 [-1.00681969 -1.10150522 -1.12539033 ... -1.17126198 -0.66844566
  -1.01747167]
 ...
 [ 0.42302505  1.57817624  2.39866392 ...  0.49704733 -0.27969611
  -0.65566573]
 [ 0.41532694 -0.3326363  -1.00681969 ... -0.46804253 -0.50898781
  -0.53788803]
 [-0.85788459 -0.91482526 -0.94094418 ... -0.2341114  -0.0505935
  -0.46615175]]
[ 0.27074327  0.68139625 -0.34976115 ... -1.3251647  -1.54211188
 -0.6024969 ]


ok, 接下来搞一下各个梯度下降的算法。

在这之前，我们还是要熟悉一下前三部

1. define function set
2. find loss function
3. find best function

function set 就使用线性回归
$$y = w * x + b$$

loss function 则是使用
$$L(w,b) = 1/2 · sum(y - (w ·x + b))^2$$

第三部则就是梯度下降。
$$∑(𝑛=1)^10▒2(𝑦 ̂^𝑛−(𝑏+𝑤∙𝑥_𝑐𝑝^𝑛 ))(−𝑥_𝑐𝑝^𝑛 ) $$

In [84]:
# 一般的梯度下降
def gd(x_train, y_train):
    l_rate        = 0.001 # 学习率
    sample_num    = x_train.shape[0]
    feature_num   = x_train.shape[1]
    w             = np.empty(feature_num)
    b             = 0.0
    epoch_num     = 300 # 训练次数
    
    for i in range(epoch_num):
        y_pre  = np.dot(x_train, w) + b # shape = (sample_num, 1)
        loss   = y_train - y_pre
        # sum 所有的 (y - y_pre)(-x) x为一个 feature, 这个操作一共要 sample_num 次
        #  如果不对其求平均，很快数据就会达到 nan
        w_grad = np.dot(loss, -x_train) / sample_num # shape = (feature, 1)
        b_grad = loss.sum() / sample_num
#         print(w_grad)
#         print(b_grad)
#         print(-x_train)
        w      -= l_rate * w_grad
        b      -= l_rate * b_grad

    return w, b
w, b = gd(x_train_set, y_train_set)
print(w)
print(b)

[ 2.72580949e-02 -1.82973941e-04  1.17848791e-02  9.10530876e-03
 -5.41954036e-04  5.36399627e-03 -7.42634255e-03 -2.36141285e-03
  8.85103091e-01]
-0.0005279219013381018
